In [1]:
from to_verb import *
from factuality_nominalizations import *
import sys, os, catvar

fact_corpus_path = "resources/unified_factuality"
factuality_dev = os.path.join(fact_corpus_path, "dev.conll")

In [3]:
ff = lambda b: df_from_table_string(b, col_names=COL_NAMES)
dfs=list(loadLineSplitFiles(factuality_dev, ff))

In [4]:
sentences = [' '.join(df['tok']) for df in dfs]
tokSents =  [list(df['tok']) for df in dfs]
find_nominalizations = lambda df: df[(df['pos'].isin(["NN", "NNS"])) & (df['factuality']!="_")]
count_nominalizations = lambda df: find_nominalizations(df).shape[0]
find_non_nominalization = lambda df: df[(df['pos'].isin(["NN", "NNS"])) & (df['factuality']=="_")]
count_non_nominalization = lambda df: find_non_nominalizations(df).shape[0]

# verbalize using both WordNet and CatVar
def get_verb_forms(nn):
    wordnet_verbs = convert_pos(nn, WN_NOUN, WN_VERB)
    catvar_verbs = catvar.catvariate(nn)
    # sort by distance
    vrbs = [v for v,w in results_by_edit_distance(nn, wordnet_verbs + catvar_verbs)]
    if vrbs:
        return vrbs, True
    else:
        return [nn], False

In [5]:
# # prompts for qasrl-crowdsourcing variant 
prompts = [(tokSent, int(nom_row['tok-id']), get_verb_forms(nom_row['tok'])[0])
           for tokSent, df in zip(tokSents, dfs) 
           for i,nom_row in find_nominalizations(df).iterrows()]
noms = [p[0][p[1]] for p in prompts]
def vis_prompts(prompts):
    return dict(enumerate([(' '.join(p[0][p[1]-2:p[1]+2]), p[2]) for p in prompts]))

In [ ]:
import json
json.dump(prompts[:25],  open("QANom_prompts_20", "w"))

In [ ]:
# prompts for MTurk UI nom. identification task
prompts_noms = [{"context": sent,
                 "intent": int(nom_row['tok-id']),
                 "noun": sent.split(" ")[int(nom_row['tok-id'])],
                 "verb_forms": ' '.join(verb_forms[:5]),
                 "is_factuality_event": "yes",
                 "is_had_auto_verbs": "yes" if is_had_verbs else "no"}
           for sent, df in zip(sentences, dfs) 
           for i,nom_row in find_nominalizations(df).iterrows()
           for verb_forms, is_had_verbs in [get_verb_forms(nom_row['tok'])]]
prompts_not_noms = [{"context": sent,
                     "intent": int(nom_row['tok-id']),
                     "noun": sent.split(" ")[int(nom_row['tok-id'])],
                     "verb_forms": ' '.join(verb_forms[:5]),
                     "is_factuality_event": "no",
                     "is_had_auto_verbs": "yes" if is_had_verbs else "no"}
           for sent, df in zip(sentences, dfs) 
           for i,nom_row in find_non_nominalization(df).iterrows()
           for verb_forms, is_had_verbs in [get_verb_forms(nom_row['tok'])]]
def vis_prompts(prompts):
    return dict(enumerate([(' '.join(p["context"].split()[p["intent"]-2:p["intent"]+2]), p["verb_forms"]) 
                           for p in prompts[:100]]))

# noms = [p["context"][p["intent"]] for p in prompts_noms]

In [ ]:
prompts = prompts_noms[:30] + prompts_not_noms[2:32]
prompts[2]

In [ ]:
# export prompts to csv (shuffled)
# prompt_df = pd.DataFrame(prompts)
# prompt_df.sample(frac=1).to_csv("nom_identification_prompts.csv", index_label="id")

In [ ]:
# manual assessment - bad nom. or verb-form - classified
problematic_wn_nomin = {
    # question with verb is not natural
    32: ('trading activity dwindled', u'act'),
    # verb not derivationally related to noun (even though meaning is presevred) - while there is a sutable verb:
    4: ('a threat to', u'menace'),
    101: ('In anticipation of', u'expect'),
    70: ('a turnaround next', u'reverse'),
    68: ('million loss for', u'pass'),
    71: ('the loss ,', u'pass'),
    121: ('a gain from', u'profit'),
    184: ('the alliance .', u'align'),
    230: ('quarter loss of', u'pass'),
    # not the correct source-verb (but same "root")
    132: ('the revolution that', u'revolutionize'),
    # not a nominalization (no verb is suitable)
    24: ('these problems are', u'trouble'),
    79: ('procurement scandal .', u'scandalise'),
    103: ('transition period .', u'point'),
    181: ('50th anniversary .', 'anniversary'),
    213: ('world events .', u'effect'),
    218: ('no problem with', u'trouble'),
    # light-verb constructions nouns (e.g. "effort" - "make an effort" would have worked, but no verb is suitable) 
    23: ('undercutting efforts to', u'exert'),
    38: ("'s efforts to", u'exert'),
    208: ('those efforts .', u'exert'),
    # not a verb!
    104: ('new series of', 'series'),
    112: ('a deadline StatesWest', 'deadline'),
    118: ('a restructuring program', 'restructuring'),
    214: ('increasingly terrorism knows', 'terrorism'),
    235: ('first salvo in', u'volley'),
    # two-word verbs 
    46: ('massive selloffs ,', u'sell_off'),
    48: ('share turnover for', u'turn_over'),
    75: ('the layoffs of', u'lay_off')
}
prb_prompts = {i:p for i,p in enumerate(prompts) if i in problematic_wn_nomin}
{i:(' '.join(p[0][p[1]-3:p[1]+3]), p[2]) for i,p in prb_prompts.iteritems()}

In [ ]:
pd.set_option('display.max_colwidth',400)
all_prompts=prompts_noms + prompts_not_noms
all_prompts_with_verbs = [p for p in all_prompts if p["is_had_auto_verbs"]=="yes"]
# pd.DataFrame(all_prompts).groupby(["is_had_auto_verbs", "is_factuality_event"]).apply(lambda df: df.sample(10))

In [ ]:
import verb_to_nom
reload(verb_to_nom)
import pattern.en
possible_noms = verb_to_nom.get_all_possible_nominalizations()

In [ ]:
len(possible_noms)

In [ ]:
pdf=pd.DataFrame(all_prompts)
pdf['in_possible_noms'] = pdf.noun.str.lower().isin(possible_noms)
pdf['is_infinitive'] = pdf.noun.str.lower().isin(verb_to_nom.all_wn_verbs)
pdf['both_filters_or'] = pdf.in_possible_noms | (pdf.is_had_auto_verbs=="yes")
pdf['both_filters_and'] = pdf.in_possible_noms & ((pdf.is_had_auto_verbs=="yes")|pdf.is_infinitive)
pdf.groupby(["both_filters_or", "is_factuality_event", ]).count()
# pdf.groupby(["is_factuality_event", "both_filters_and"]).apply(lambda p: p.sample(15))
# pdf.groupby(["is_factuality_event", "is_had_auto_verbs"]).count()
# pdf[(pdf.is_factuality_event=="yes") & (~pdf.in_possible_noms) & (pdf.is_had_auto_verbs=="yes")].sample(10)
# pdf[(pdf.is_factuality_event=="no") & (pdf.both_filters_and) ].sample(30)

In [ ]:
chosen_prompts=pdf[pdf.both_filters_and][[u'context', u'intent', u'is_factuality_event', u'is_had_auto_verbs',
       u'noun', u'verb_forms', u'in_possible_noms']].sample(60)
# chosen_prompts.to_csv("nom_identification_prompts.csv", index_label="id")
chosen_prompts

In [ ]:
pdf[pdf.both_filters_and & (pdf.is_had_auto_verbs=="no")]z

# Exploring about unified_factuality dataset 
Investigate the potential to replicate Kenton Lee annotation methodology to gather "event detection", but only for nouns. Is it a good way to annotate Nom.Ident. data?

In [86]:
NN_POS = ['NN', 'NNS', 'NNP']
from collections import defaultdict
pos2predicates = defaultdict(set)
df=dfs[1]
examples = []
for df in dfs:
    tokSent = list(df['tok'])
    sent = ' '.join(tokSent)
    for i,r in df.iterrows():
        if r.factuality not in ["Nan", "nan", "_"]:
            pos2predicates[r.pos].add(r.lemma)
            if r.pos in NN_POS:
                examples.append((r.tok, r['tok-id'], sent))

In [88]:
distr={k:len(v) for k,v in pos2predicates.iteritems()}
len(dfs)

1956

In [80]:
import imp, sys
sys.path.append('../myJupyterUtils/')
import utils

utils.printmd("***hhhh***")

<span style='color:None'>***hhhh***</span>